In [1]:
import requests
import json
import pandas as pd
import datetime
from dateutil.tz import gettz

In [2]:
#request할 역의 list를 담은 file load
stationInfo = pd.read_csv("/content/2019_line1_.csv") #You should check your file path
stationInfo #checking 

,역명,승하차총승객수
0,서울,43842488
1,영등포,37413027
2,수원,35250100
3,청량리(서울시립대입구),32077489
4,용산,31533761
...,...,...
98,봉명,967138
99,배방,791794
100,직산,698085
101,옥수,32440


In [3]:
temp = stationInfo['역명']
print(temp[:30])

0               서울
1              영등포
2               수원
3     청량리(서울시립대입구)
4               용산
5               종각
6               부천
7               부평
8             종로3가
9               송내
10              역곡
11              안양
12              금정
13            종로5가
14              시청
15              개봉
16              주안
17             의정부
18              구로
19             제기동
20              동암
21       왕십리(성동구청)
22         가산디지털단지
23             동인천
24              병점
25              독산
26            성균관대
27             신설동
28             동대문
29              대방
Name: 역명, dtype: object


In [4]:
ServiceKeyList = ["524e6361756a75613739536b6a7a54"] #서비스 키 list
KEY = '/' + ServiceKeyList[0] 
TYPE = '/' + "json"
SERVICE = '/' + "realtimeStationArrival"
START_INDEX = '/' + '0'
END_INDEX = '/' + '10'

for idx in range(30):
  #search할 target station을 list로 만들어서 크롤링 할 것
  #if list가 큰 경우 file로 만들어서 load/save 할 것 
  statnNm = '/' + temp[idx] 

  url = "http://swopenAPI.seoul.go.kr/api/subway" + KEY + TYPE + SERVICE + START_INDEX + END_INDEX + statnNm
  
  response = requests.get(url)
  #response가 제대로 왔는지 checking
  if response.status_code is not 200:
    print("response Error")
    exit()
  
  jsonData = response.json()

  try: #request에 성공한 station만을 추출하여 json 불러옴
    df = pd.json_normalize(jsonData['realtimeArrivalList'])
    print(statnNm)
  except:
    continue

  currTime = datetime.datetime.now(gettz("Asia/Seoul"))  #한국 시간대 사용
  df = pd.DataFrame(df)

  fileName = (str(currTime.year) + '-' + str(currTime.month) + '-' +
  str(currTime.day) + " " + str(currTime.hour) + ':' + 
  str(currTime.minute) + ':' + str(currTime.second) + " " + 
  statnNm.replace('/', '') + ".csv")

  df.to_csv(fileName, mode='w')

/서울
/영등포
/수원
/용산
/종각
/부천
/부평
/종로3가
/송내
/역곡
/안양
/금정
/종로5가
/시청
/개봉
/주안
/의정부
/구로
/제기동
/동암
/가산디지털단지
/동인천
/병점
/독산
/성균관대
/신설동
/동대문
/대방
